In [1]:
import pandas as pd
import numpy as np

In [2]:
commuter = pd.read_csv('./output_commuters/person.csv')
non_commuter = pd.read_csv('./output_non_commuters/person.csv')

In [3]:
person = pd.concat([commuter, non_commuter], axis=0, ignore_index=True)

In [4]:
person.to_csv('./output/person.csv')

In [5]:
activity = pd.DataFrame(columns=['sampno', 'perno', 'activity_type', 'start_time', 'end_time', 'location'])

for index, row in person.iterrows():
    state_sequence = eval(row['state_sequence'])
    current_activity = state_sequence[0]
    start_time = 0
    actno = 1
    for time, state in enumerate(state_sequence):
        if state == 'other1':  # start of a new 'other' activity
            state = 'other'
            activity_change = True
        else:
            activity_change = state != current_activity

        if activity_change or time == len(state_sequence) - 1:  # activity changed or end of day
            end_time = time
            location = row['home_tract'] if current_activity == 'home' else row['work_tract'] if current_activity == 'work' else ''
            new_row = pd.DataFrame({
                'sampno': [row['sampno']], 
                'perno': [row['perno']], 
                'actno': [actno],
                'activity_type': [current_activity], 
                'start_time': [start_time], 
                'end_time': [end_time], 
                'location': [location]
            })
            activity = activity.append(new_row, ignore_index=True)
            start_time = time
            current_activity = state
            if activity_change:
                actno += 1


C:\Users\15123\AppData\Local\Temp\ipykernel_4336\4202422649.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activity = activity.append(new_row, ignore_index=True)


In [6]:
# 对sampno、perno和start_time进行排序
activity.sort_values(by=['sampno', 'perno', 'start_time'], inplace=True)

# 对每个sampno和perno组合计算actno
activity['actno'] = activity.groupby(['sampno', 'perno']).cumcount() + 1

# 将actno列移动到第三列
activity = activity[['sampno', 'perno', 'actno', 'activity_type', 'start_time', 'end_time', 'location']]

In [7]:
activity[activity['sampno']==1123408]

,sampno,perno,actno,activity_type,start_time,end_time,location
17736,1123408,1,1,home,0,144,462100.0
17737,1123408,1,2,other,144,163,
17738,1123408,1,3,other,163,221,
17739,1123408,1,4,other,221,240,
17740,1123408,1,5,home,240,288,462100.0
82,1123408,2,1,home,0,96,462100.0
83,1123408,2,2,work,96,151,139600.0
84,1123408,2,3,other,151,161,
85,1123408,2,4,other,161,168,
86,1123408,2,5,other,168,220,


In [8]:
# rank lookup matrix
CT = pd.read_csv('Census_Tract_Locations__LA_.csv',delimiter = ',')
Points = pd.read_csv('Points.csv',delimiter = ',')

from sklearn.neighbors import NearestNeighbors
import geopandas as gpd
import numpy as np

# Create geopandas GeoDataFrames
gdf_ct = gpd.GeoDataFrame(CT, geometry=gpd.points_from_xy(CT.Longitude, CT.Latitude))
gdf_points = gpd.GeoDataFrame(Points, geometry=gpd.points_from_xy(Points.lon, Points.lat))

# Prepare the data for NearestNeighbors method
points_coord = np.array(list(gdf_points.geometry.apply(lambda geom: (geom.x, geom.y))))
ct_coord = np.array(list(gdf_ct.geometry.apply(lambda geom: (geom.x, geom.y))))

# Build the tree
nbrs = NearestNeighbors(n_neighbors=len(gdf_points), algorithm='ball_tree').fit(points_coord)

# Query the tree
distances, indices = nbrs.kneighbors(ct_coord)

# Create the ranking matrix
Rank = pd.DataFrame(indices, columns=gdf_points.id, index=gdf_ct['Tract Number']).apply(lambda x: x.rank(), axis=1)

In [9]:
Rank = Rank + 1
Rank

id,9480,9481,9482,9483,23176,23177,23178,23179,23180,23181,...,26062,26063,26064,26065,26066,26067,26068,26069,26070,26071
Tract Number,,,,,,,,,,,,,,,,,,,,,
101110,7.0,5.0,3.0,115.0,12.0,119.0,122.0,6.0,2.0,129.0,...,25265.0,25255.0,24335.0,25261.0,24382.0,24384.0,24890.0,24892.0,24889.0,20503.0
101122,12.0,11.0,13.0,6.0,8.0,5.0,15.0,7.0,26439.0,10.0,...,25265.0,24339.0,25261.0,24382.0,24335.0,24384.0,24890.0,24892.0,24889.0,20503.0
101210,54.0,58.0,4.0,3.0,5.0,2.0,130.0,18.0,7.0,6.0,...,25633.0,24335.0,25255.0,25261.0,24382.0,24384.0,24890.0,24892.0,24889.0,20503.0
101220,18.0,26.0,46.0,17.0,45.0,32.0,40.0,56.0,4.0,34.0,...,25637.0,24339.0,24382.0,25633.0,24335.0,24384.0,24890.0,24892.0,24889.0,20503.0
101300,38.0,39.0,25.0,51.0,42.0,20.0,32.0,30.0,36.0,43.0,...,24382.0,24384.0,25637.0,24339.0,25633.0,24335.0,24890.0,24892.0,24889.0,20503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573902,19646.0,19649.0,19643.0,19644.0,19647.0,19641.0,19645.0,26213.0,18919.0,19648.0,...,24895.0,24893.0,24896.0,25632.0,25637.0,24891.0,25633.0,24890.0,24892.0,24889.0
576602,20255.0,20245.0,20265.0,20280.0,20241.0,20251.0,20232.0,20290.0,20243.0,20250.0,...,24893.0,24895.0,24896.0,25637.0,25632.0,25633.0,24891.0,24890.0,24892.0,24889.0
599100,20503.0,8585.0,8588.0,8603.0,8592.0,8589.0,8618.0,8596.0,8590.0,8621.0,...,24896.0,24401.0,24388.0,24892.0,24384.0,24382.0,24895.0,24891.0,24889.0,24383.0


In [49]:
# rank-based probability matrix generation
# Apply the power function
Prob = Rank.pow(-0.41)

# Normalize the rows to sum to 1
Prob = Prob.div(Prob.sum(axis=1), axis=0)

In [99]:
# Rank-based location choice
point_dict = dict(zip(Points['id'], Points['CT']))
activity['points'] = ''
for index, row in activity.iterrows():
    if row['activity_type'] == 'other':
        prev_row = activity.loc[index - 1]
        tract = prev_row['location']
        tract_prob = Prob.loc[tract]
        point = np.random.choice(tract_prob.index, p=tract_prob.values)
        activity.at[index, 'points'] = point
        activity.at[index, 'location'] = point_dict.get(point)

In [108]:
activity['location'] = activity['location'].astype(int)

def time_transfer(time):
    minute = time * 5
    hours, minutes = divmod(minute, 60)
    time_str = f"{hours:02d}:{minutes:02d}"
    return time_str

activity['start'] = activity['start_time']
activity['end'] = activity['end_time']
activity['start'] = activity['start'].apply(time_transfer)
activity['end'] = activity['end'].apply(time_transfer)

In [109]:
activity

,sampno,perno,actno,activity_type,start_time,end_time,location,points,start,end
17421,1032558,1,1,home,0,108,242300,,00:00,09:00
17422,1032558,1,2,other,108,175,271801,1896,09:00,14:35
17423,1032558,1,3,other,175,189,462400,10199,14:35,15:45
17424,1032558,1,4,other,189,240,192700,1914,15:45,20:00
17425,1032558,1,5,home,240,288,242300,,20:00,24:00
...,...,...,...,...,...,...,...,...,...,...
66175,7211146,5,2,other,140,240,212102,53,11:40,20:00
66176,7211146,5,3,home,240,288,503601,,20:00,24:00
66177,7211146,6,1,home,0,189,503601,,00:00,15:45
66178,7211146,6,2,other,189,241,197300,9825,15:45,20:05


In [110]:
activity.to_csv('./output/activity.csv',index = False)